The first stage to running an encryption 

In [6]:
import sys
!{sys.executable} -m pip install pyserial

    100% |████████████████████████████████| 92kB 550kB/s ta 0:00:01    22% |███████▎                        | 20kB 2.7MB/s eta 0:00:01


In [7]:
import time
import serial
from time import sleep

from pynq.overlays.base import BaseOverlay
from pynq.lib.arduino import arduino_io

base = BaseOverlay('base.bit')

# trigger for the pynq on arduino pins a0 and a1
trigger_out = arduino_io.Arduino_IO(base.iop_arduino.mb_info, 14, 'out')
trigger_in = arduino_io.Arduino_IO(base.iop_arduino.mb_info, 15, 'in')

key = b'aesEncryptionKey'  # can change this

# pull in the customlibrary
%run pyaes.ipynb

In [8]:
led0 = base.leds[0] #Corresponds to LED LD0
led1 = base.leds[1] #Corresponds to LED LD1
led2 = base.leds[2] #Corresponds to LED LD2
led3 = base.leds[3] #Corresponds to LED LD3

In [9]:
sw0 = base.switches[0] #Corresponds to SW0
sw1 = base.switches[1] #Corresponds to SW1

In [10]:
btn0 = base.buttons[0] # button 0 hopefully
btn1 = base.buttons[1]
btn2 = base.buttons[2]
btn3 = base.buttons[3]

In [11]:
class CWSerial:
    def __init__(self, device):
        self.device = device
        self.connection = None

    def configure(self):
        self.connection = serial.Serial(
            self.device, 38400, timeout=5, bytesize=serial.EIGHTBITS, parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE, xonxoff=False)
        time.sleep(1)

    def read():
        return self.connection.readline().strip()

cw_com = CWSerial("/dev/ttyUSB0")

In [12]:
print("Welcome to the pynq z2 board test")
mode = 0 # mode 0 is regular input, mode 1 is file input

Welcome to the pynq z2 board test


The setup is now complete and we can begin encryptions.

In [18]:
def encrypt(key, plaintext):
    print(f"Encrypting {plaintext} with key {key}")
    encrypter = Encrypter(AESModeOfOperationECB(key))
    ciphertext = encrypter.feed(plaintext)
    ciphertext += encrypter.feed()

    #            print( repr(ciphertext))
    for letter in ciphertext:
        print('{:0>2}'.format(str(hex(letter))[2:]), end='')
    return ciphertext

In [ ]:

if not mode:
    # switch based LED toggling for test
    data= ''
    key = b'aesEncryptionKey'
    while(True):  # All the code below while(True) runs forever
        if btn0.read():
            print("SW1 controls whether lights and encryption or cw encryption")
            print("In encryption mode:")
            print("\t BTN0 resets the data")
            print("\t BTN1 orders input of data\n")
            print("\t BTN2 allows key input (padded to 16 characters)")
            print("\t BTN3 does encryption")
                
            print("Sample encryption and help display:")
            key = b'aesEncryptionKey'
            plaintext = b'hello, world0000'

            aes = AESModeOfOperationECB(key)
            ciphertext = aes.encrypt(plaintext)

            print( repr(ciphertext))
            decrypted = aes.decrypt(ciphertext)
            print (f"match? {decrypted == plaintext}")

        elif not sw1.read():
            led2.toggle()
            led3.toggle()
            if btn1.read():
                led1.toggle()
                led0.toggle()
                trigger_out.write(1)
                sleep(1)
                trigger_out.write(0)
                data = input()
                x = encrypt(key, str.encode(data))
                print(x)

            
        # run the input read and encryption
        else:
            if trigger_in.read():
                trigger_out.write(1)
                sleep(.25)
                trigger_out.write(0)
                data = cw_com.read()
                print(f'data to encrypt {data}')
                x = encrypt(key, str.encode(data))
                print(x)
            
#                key = key.encode()

hello
Encrypting b'hello' with key b'aesEncryptionKey'
0570e2d435a33b2d34e7b096a1f1ce53b'\x05p\xe2\xd45\xa3;-4\xe7\xb0\x96\xa1\xf1\xceS'
